In [1]:
import itertools
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.csgraph import shortest_path
import math
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

In [2]:
random.seed(0)
# 広告集合
A = [i for i in range(3)]

# ユーザー集合
U = [i for i in range(5)]

# ユーザーの関係(枝)
E = []
for a in range(len(A)):
    E_i = [[0.0 for i in range(len(U))] for j in range(len(U))]
    for i in range(len(U)):
        for j in range(i+1,len(U)):
            if i != j:
                if random.uniform(0,1) < 0.6:
                    E_i[i][j] = 1
                    E_i[j][i] = 1
    E.append(E_i)

# engagementにつき広告主が払う金額
r = [random.randint(1,5) for i in range(len(A))]

# ユーザーuに対する広告シーケンスの戦略空間
Sigma_u = [list(itertools.permutations(A)) for i in range(len(U))]

# ユーザーuに割り当てられる広告シーケンス sigma_u([ユーザー][割り当てた広告シーケンス])
#sigma_u

# ユーザーuが広告iを見た後に広告iを共有する確率 q[ユーザー][広告]
q = [[random.randint(0,100) / 100 for j in range(len(A))] for i in range(len(U))]

# ユーザーuが広告iを見た後に次の広告を見る確率 c[ユーザー][広告]
c = [[random.randint(0,100) / 100 for j in range(len(A))] for i in range(len(U))]

In [3]:
r

[4, 4, 5]

In [4]:
Sigma_u

[[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)],
 [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)],
 [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)],
 [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)],
 [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]]

In [5]:
E

[[[0.0, 0.0, 0.0, 1, 1],
  [0.0, 0.0, 1, 1, 0.0],
  [0.0, 1, 0.0, 1, 1],
  [1, 1, 1, 0.0, 1],
  [1, 0.0, 1, 1, 0.0]],
 [[0.0, 0.0, 1, 1, 0.0],
  [0.0, 0.0, 0.0, 1, 0.0],
  [1, 0.0, 0.0, 0.0, 0.0],
  [1, 1, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 1, 0.0, 0.0, 0.0],
  [1, 0.0, 1, 1, 1],
  [0.0, 1, 0.0, 0.0, 0.0],
  [0.0, 1, 0.0, 0.0, 0.0],
  [0.0, 1, 0.0, 0.0, 0.0]]]

In [6]:
q

[[0.33, 0.07, 0.7],
 [0.01, 0.11, 0.92],
 [0.51, 0.9, 1.0],
 [0.85, 0.8, 0.0],
 [0.78, 0.63, 0.42]]

In [7]:
c

[[0.31, 0.93, 0.41],
 [0.9, 0.08, 0.24],
 [0.72, 0.28, 0.3],
 [0.18, 0.69, 0.57],
 [0.11, 0.1, 0.4]]

In [8]:
# ユーザーuがスロットk(スロット0~|A|-1)まで見る確率
u = 2
k = 1
sigma_u = Sigma_u[u][2]
print("ユーザー:{}\n{}個見る\n広告シーケンス{}".format(u,k,sigma_u))
def view_ad_probability_k(c, u, k, sigma_u):
    pi_c = 1
    for t in range(0,k):
        pi_c *= c[u][sigma_u[t]]
    return pi_c
print(view_ad_probability_k(c, u, k, sigma_u))

ユーザー:2
1個見る
広告シーケンス(1, 0, 2)
0.28


In [9]:
# ユーザーuが広告sigma_u[k]までを見て、共有する確率
u = 2
k = 1
sigma_u = Sigma_u[u][2]
print("ユーザー:{}\n{}個目を共有する\n広告シーケンス{}".format(u,k,sigma_u))
def share_ad_probability_k(c, q, u, k, sigma_u):
    print(q[u][sigma_u[k]])
    return q[u][sigma_u[k]] * view_ad_probability_k(c, u, k, sigma_u)
print(share_ad_probability_k(c, q, u, k, sigma_u))

ユーザー:2
1個目を共有する
広告シーケンス(1, 0, 2)
0.51
0.1428


# Independ Cascade Model

In [10]:
random.seed(0)
# p:枝の確率集合
# p(E)
p = []
for a in range(len(A)):
    # p_i:広告iの枝確率
    p_i = [[0.0 for i in range(len(U))] for j in range(len(U))]
    for i in range(len(U)):
        for j in range(i+1,len(U)):
            if i != j:
                p_i[i][j] = E[a][i][j] * random.randint(1,100) / 100
                p_i[j][i] = E[a][j][i] * random.randint(1,100) / 100
    p.append(p_i)

In [11]:
p

[[[0.0, 0.0, 0.0, 0.34, 0.63],
  [0.0, 0.0, 0.39, 0.46, 0.0],
  [0.0, 0.62, 0.0, 0.18, 0.18],
  [0.66, 0.75, 0.37, 0.0, 0.13],
  [0.52, 0.0, 0.97, 0.8, 0.0]],
 [[0.0, 0.0, 0.91, 0.19, 0.0],
  [0.0, 0.0, 0.0, 0.43, 0.0],
  [0.78, 0.0, 0.0, 0.0, 0.0],
  [0.4, 0.61, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.71, 0.0, 0.0, 0.0],
  [0.62, 0.0, 0.12, 0.52, 0.86],
  [0.0, 0.93, 0.0, 0.0, 0.0],
  [0.0, 0.91, 0.0, 0.0, 0.0],
  [0.0, 0.81, 0.0, 0.0, 0.0]]]

In [12]:
E

[[[0.0, 0.0, 0.0, 1, 1],
  [0.0, 0.0, 1, 1, 0.0],
  [0.0, 1, 0.0, 1, 1],
  [1, 1, 1, 0.0, 1],
  [1, 0.0, 1, 1, 0.0]],
 [[0.0, 0.0, 1, 1, 0.0],
  [0.0, 0.0, 0.0, 1, 0.0],
  [1, 0.0, 0.0, 0.0, 0.0],
  [1, 1, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 1, 0.0, 0.0, 0.0],
  [1, 0.0, 1, 1, 1],
  [0.0, 1, 0.0, 0.0, 0.0],
  [0.0, 1, 0.0, 0.0, 0.0],
  [0.0, 1, 0.0, 0.0, 0.0]]]

In [13]:
# 1であるところだけ分岐して順列を全列挙
def permutation_01(l):
    l_result = []
    # パターン数:n
    n = int(sum(l))
    zero = np.array([0 for i in range(2**n)])
    l_01 = np.array(list(itertools.product([0,1], repeat=n)))
    l_01_T = l_01.T
    j = 0
    for i in l:
        if i == 0:
            l_result += [zero]
        else:
            l_result += [l_01_T[j]]
            j += 1
    return np.array(l_result).T

# permutation_01([0,1,0,1,1])
# array([[0, 0, 0, 0, 0],
#        [0, 0, 0, 0, 1],
#        [0, 0, 0, 1, 0],
#        [0, 0, 0, 1, 1],
#        [0, 1, 0, 0, 0],
#        [0, 1, 0, 0, 1],
#        [0, 1, 0, 1, 0],
#        [0, 1, 0, 1, 1]])

In [14]:
# ネットワークの全パターンlive_edgeグラフの作成
# live_edge_set(枝集合)
def live_edge_set(E_i):
    l_list = []
    for l in E_i:
        l_list.append(permutation_01(l))
    return np.array(list(itertools.product(*l_list))) 

# live_edge_set(E[0])
# array([[[0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0]],
#
#        [[0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0],
#         [0, 0, 0, 1, 0]],
#
#        ...,

In [15]:
# そのグラフになる確率
# live_edge_prob(元ネットワーク, live_edgeグラフ, 枝確率)
def live_edge_prob(original, live_edge, p):
    n = len(live_edge)
    # 枝がある確率
    prob_T = live_edge * p
    prob_T = np.where(prob_T == 0, 1, prob_T)
    # 総積を求める
    prob_T = np.prod(prob_T)
    
    # 枝がない確率
    prob_F = (original - live_edge) * (np.ones((n,n))-p)
    prob_F = np.where(prob_F == 0, 1, prob_F)
    # 総積を求める
    prob_F = np.prod(prob_F)
    
    return prob_T * prob_F

# live_edge_prob(E[0], live_edge_set(E[0])[10000], p[0])


# 確率計算のため、数が小さくなっていくためlogで計算する必要があるかもしれない

In [16]:
# live_edgeグラフ上でシードから到達可能な頂点数
def reachable_num(live_edge, seed):
    # live_edgeグラフの各頂点までの最短距離を求める
    short_path = shortest_path(live_edge)
    
    # inf(到達できない)を0に変換している
    short_path = np.where(np.float('inf') == short_path, 0, short_path)
    
    # seedの到達可能な頂点をor演算することで重複を消している
    reachable_node_num = np.logical_or.reduce(short_path[seed])
    
    # seedの頂点をFalseにしている
    reachable_node_num[seed] = False
    
    # seedの頂点数を足して出力
    return len(set(seed)) + sum(reachable_node_num)

# reachable_num(live_edge_set(E[0])[10002], [0,1])
# 5

In [17]:
# 影響数の期待値
# expected_influence_num(ネットワーク, 枝確率, シード)
def expected_influence_num(E, p, seed):
    # live_edgeグラフを全列挙
    live_edge_sets = live_edge_set(E)
    
    expected_num = 0
    for l_e_s in tqdm(live_edge_sets):
        # そのlive_edgeグラフになる確率 * そのグラフ上でシードからの到達頂点数の総和
        expected_num += live_edge_prob(E, l_e_s, p) * reachable_num(l_e_s, seed)
    return expected_num

# for i in range(5):
#     print(expected_influence_num(E[0], p[0], [i]))
# 3.710841757347267
# 2.7803227401623367
# 2.9310440468303938
# 3.8615958011324687
# 4.544981497080014

In [18]:
# Independet Cascade Modelにおいて、考えられるlive_edgeグラフの枝集合を返す
# sub_gragh_edge(枝確率集合)
def live_edge_gragh_edges(p):
    prop_edge = []
    for e_prop_set in p:
        prop_edge_temp = []
        for e_prop in e_prop_set:
            if e_prop > random.random():
                prop_edge_temp.append(1)
            else:
                prop_edge_temp.append(0)
        prop_edge.append(prop_edge_temp)
    return prop_edge

# live_edge_gragh_edges(p[0])
# [[0, 0, 0, 1, 0],
#  [0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0],
#  [1, 0, 1, 0, 0],
#  [1, 0, 1, 0, 0]]

$ T = \frac{n^2}{\epsilon^2}\ln{\frac{1}{\delta}} $回繰り返すと、$\sigma(A)$は少なとも$1-\delta$の確率で$(1\pm\epsilon)$近似を達成する

In [19]:
# 影響数の期待値(近似)
# approxim_expected_influence_num(枝確率, シード, ε, δ)
def approxim_expected_influence_num(p, seed, epsi, delta):
    # 頂点数
    n = len(p)
    # 試行回数を算出する
    T = int(((n**2) / (epsi**2)) * np.log(1/delta)) + 1
    
    # 各回のシュミレーションの結果の和が格納される
    X = 0
    # T回シュミレーションしていく
    for i in tqdm(range(T)):
        # live_edgeグラフを作る
        live_edge_sets = np.array(live_edge_gragh_edges(p))
        X += reachable_num(live_edge_sets, seed)
    expected_num = X / T
    return expected_num

In [20]:
epsi = 0.01
delta = 0.1
int(((5**2) / (epsi**2)) * np.log(1/delta)) + 1

575647

近似値と厳密値を比べる

In [21]:
print(approxim_expected_influence_num(p[0], [0], 0.01, 0.1))


3.7116826805316454


In [22]:
print(expected_influence_num(E[0], p[0], [0]))


3.710841757347267


頂点数が少ないと、厳密に求めた方が高速であるが、近似の利点は頂点数が増えてもεとδはそのままであれば、実行回数が変わらない点である。

In [54]:
# C_i(seed):広告i、シードseedのときのユーザーの影響数の期待値
# I_i(seed):シードによって得られる広告iの期待収益
# expected_revenue(エンゲージメントごとに払う金額, 枝集合, 確率枝集合, シード)
def expected_revenue(r, E, p, seed):
    return r * expected_influence_num(E, p, seed)

# expected_revenue(r[0], E[0], p[0], [0,1])
# 17.490684396302193

# 問題設定

In [69]:
# V:到着したユーザー集合
# _S_:到着したユーザー集合に割り当てた広告シーケンス
# p_i_u_sigma_u:_S_のもとでuがiを共有する確率

# # 参考
# u = 1
# i = 1
# sigma_u = Sigma_u[u][3]
# p_i_u_sigma_u = share_ad_probability_k(c, q, u, i, sigma_u)
# 0.92